In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

## Data Ingestion

In [3]:
from langchain_community.document_loaders import TextLoader

In [4]:
loader = TextLoader(r'/Users/atharvabot7/Downloads/MLOPS-Tutorials/LLMOps-Project/Agentic_AI.txt', encoding='utf8')

In [5]:
documents = loader.load()

In [6]:
documents

[Document(metadata={'source': '/Users/atharvabot7/Downloads/MLOPS-Tutorials/LLMOps-Project/Agentic_AI.txt'}, page_content='Agentic AI represents the next strategic inflection point in artificial intelligence—moving systems from passive responders to proactive, goal-driven entities capable of taking initiative. Unlike traditional models that wait for explicit prompts, agentic AI systems can reason about tasks, plan multi-step actions, and autonomously decide which tools or information sources to use. This shift unlocks far more flexible automation, enabling AI systems to function less like calculators and more like adaptive digital operators.\n\nThe true power of agentic AI lies in its ability to orchestrate complex workflows. An agentic system can break down a high-level objective into smaller tasks, gather the necessary data, call external APIs, or trigger downstream services—while continuously evaluating its own progress. These capabilities make agentic AI especially useful in domain

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 20)

In [9]:
text_chunks = text_splitter.split_documents(documents)

In [10]:
text_chunks

[Document(metadata={'source': '/Users/atharvabot7/Downloads/MLOPS-Tutorials/LLMOps-Project/Agentic_AI.txt'}, page_content='Agentic AI represents the next strategic inflection point in artificial intelligence—moving systems from passive responders to proactive, goal-driven entities capable of taking initiative. Unlike'),
 Document(metadata={'source': '/Users/atharvabot7/Downloads/MLOPS-Tutorials/LLMOps-Project/Agentic_AI.txt'}, page_content='initiative. Unlike traditional models that wait for explicit prompts, agentic AI systems can reason about tasks, plan multi-step actions, and autonomously decide which tools or information sources to'),
 Document(metadata={'source': '/Users/atharvabot7/Downloads/MLOPS-Tutorials/LLMOps-Project/Agentic_AI.txt'}, page_content='sources to use. This shift unlocks far more flexible automation, enabling AI systems to function less like calculators and more like adaptive digital operators.'),
 Document(metadata={'source': '/Users/atharvabot7/Downloads/MLOPS

## Deciding on Which Model to Use

In [11]:
import os
from google import genai
from langchain_core.embeddings import Embeddings
from typing import List

In [12]:
class GeminiEmbeddings(Embeddings):
    def __init__(self, model="models/text-embedding-004"):
        self.model = model
        self.client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    def _embed_one(self, text: str) -> List[float]:
        """Embed a single text string."""
        try:
            resp = self.client.models.embed_content(
                model=self.model,
                contents=text
            )
            return resp.embeddings[0].values
        except Exception as e:
            print(f"Error embedding text: {e}")
            raise

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed a list of documents."""
        return [self._embed_one(t) for t in texts]

    def embed_query(self, query: str) -> List[float]:
        """Embed a single query string."""
        return self._embed_one(query)

In [13]:
from langchain_community.vectorstores import FAISS

embeddings = GeminiEmbeddings()

vectorstore = FAISS.from_documents(text_chunks, embeddings)

In [14]:
vectorstore

## Checking if RAG is Working or not

In [15]:
query = "What is the true power of Agentic AI?"
docs = vectorstore.similarity_search(query, k=3)

In [16]:
for i, doc in enumerate(docs):
    print(f"Document {i+1}:\n{doc.page_content}\n")

Document 1:
The true power of agentic AI lies in its ability to orchestrate complex workflows. An agentic system can break down a high-level objective into smaller tasks, gather the necessary data, call external

Document 2:
Agentic AI represents the next strategic inflection point in artificial intelligence—moving systems from passive responders to proactive, goal-driven entities capable of taking initiative. Unlike

Document 3:
data, call external APIs, or trigger downstream services—while continuously evaluating its own progress. These capabilities make agentic AI especially useful in domains like customer service,



In [17]:
from langchain_core.prompts import ChatPromptTemplate

template = """ 
You are a helpful assistant for question answering tasks. 
Use the following context to answer the question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Keep your answers concise and to the point.

Question: {question}
Context: {context}
Answer:
"""

In [18]:
prompt = ChatPromptTemplate.from_template(template)

In [19]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=" \nYou are a helpful assistant for question answering tasks. \nUse the following context to answer the question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\nKeep your answers concise and to the point.\n\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [20]:
from langchain_core.output_parsers import StrOutputParser

In [21]:
output_parser = StrOutputParser()